# Introduction
![dash-logo](images/dash-logo.jpg)

**Making a dashboard** is a great way to share the value of a data science project with users––coders and noncoders alike. 

Be sure to check out the [Dash Gallery](https://dash-gallery.plotly.host/Portal/) for inspiration!

Today we will be building a basic multi-page dash application using Illinois population data and boilerplate code from [Dash Bootstrap Components](http://dash-bootstrap-components.opensource.faculty.ai/)

The code will be run with py files in Terminal, but a walkthrough of the code is available in this notebook.

# Import Libraries

In [63]:
### Data
import pandas as pd
import pickle

### Graphing
import plotly.graph_objects as go

### Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input

# Import Data

In [38]:
df = pd.read_csv('population_il_cities.csv')

df.head(2)

,Unnamed: 0,"Abingdon city, Illinois","Addieville village, Illinois","Addison village, Illinois","Adeline village, Illinois","Albany village, Illinois","Albers village, Illinois","Albion city, Illinois","Aledo city, Illinois","Alexis village, Illinois",...,"Worden village, Illinois","Worth village, Illinois","Wyanet village, Illinois","Wyoming city, Illinois","Xenia village, Illinois","Yale village, Illinois","Yates City village, Illinois","Yorkville city, Illinois","Zeigler city, Illinois","Zion city, Illinois"
0,2010-01-01 00:00:00,3286,252,37113,85,903,1183,1996,3640,830,...,1072,10798,984,1424,382,86,691,17041,1803,24421
1,2011-01-01 00:00:00,3272,253,37294,84,901,1185,1976,3617,828,...,1075,10831,977,1391,379,86,687,17295,1798,24383


The data for this project is formatted to be a datetime index. 

csv files always replace the index of files with a standard range index.

Let's reset the index.

In [46]:
df.set_index(time.iloc[:,0], drop = True, inplace = True)

df.head(2)

df = df.iloc[:,1:]

df.head(2)

,"Abingdon city, Illinois","Addieville village, Illinois","Addison village, Illinois","Adeline village, Illinois","Albany village, Illinois","Albers village, Illinois","Albion city, Illinois","Aledo city, Illinois","Alexis village, Illinois","Algonquin village, Illinois",...,"Worden village, Illinois","Worth village, Illinois","Wyanet village, Illinois","Wyoming city, Illinois","Xenia village, Illinois","Yale village, Illinois","Yates City village, Illinois","Yorkville city, Illinois","Zeigler city, Illinois","Zion city, Illinois"
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,3286,252,37113,85,903,1183,1996,3640,830,30079,...,1072,10798,984,1424,382,86,691,17041,1803,24421
2011-01-01 00:00:00,3272,253,37294,84,901,1185,1976,3617,828,30027,...,1075,10831,977,1391,379,86,687,17295,1798,24383
2012-01-01 00:00:00,3242,253,37393,84,893,1175,1992,3578,825,29956,...,1067,10864,970,1369,379,86,683,17544,1789,24327
2013-01-01 00:00:00,3228,249,37483,83,891,1166,1980,3615,818,30394,...,1059,10880,964,1360,374,85,682,17832,1772,24340
2014-01-01 00:00:00,3217,247,37426,82,886,1164,1964,3571,816,30328,...,1057,10867,957,1335,371,85,678,18174,1765,24294


# Graphing

Plotly is the designated graphing library for Dash, and offers a lot of fun interactivity. 

Below, we will create a general workflow for inputing a city name, and returning a time series of that city's population

In [47]:
labels = time.index
values = time.iloc[:,0].values
fig = go.Figure([go.Scatter(x = labels, y = values)])
fig.show()

# Dash Elements

Great! Now let's build some elements for this dashboard. 

First we will start witha dropdown menu. This will allow users to search through the available list of cities.

[Dropdown menus](https://dash.plot.ly/dash-core-components/dropdown) are formatted as a list of dictionaries. Each dictionary has two keys. 'label', and 'value.

**label:** Whatever value you want the user to see in the dropdown menu
**value:** Whatever value is needed to filter your dataset based on the users selections

Since this project is only focused on the state of Illinois, It's a bit overkill to have ', Illinois' after every single city. 

Using list comprehension we will create a list of dictionaries where the label does not have ', Illinois', but the corresponding value does.

In [52]:
names = time.columns

dropdown = [{'label':x.replace(', Illinois', ''), 'value': x} for x in names]

In [64]:
dropdown[:5]

[{'label': 'Abingdon city', 'value': 'Abingdon city, Illinois'},
 {'label': 'Addieville village', 'value': 'Addieville village, Illinois'},
 {'label': 'Addison village', 'value': 'Addison village, Illinois'},
 {'label': 'Adeline village', 'value': 'Adeline village, Illinois'},
 {'label': 'Albany village', 'value': 'Albany village, Illinois'}]

That looks great! Save the list using pickle so we can load it into our app file.

In [62]:
with open('dropdown', 'wb') as fp:
    pickle.dump(dropdown, fp)